In [1]:
import os
import json
import shutil
import random
from glob import glob
from pathlib import Path
import pandas as pd # 데이터 확인용 (선택 사항)

# --- 1. 기본 경로 설정 ---
# 이 노트북 파일(fruit_quality_project.ipynb)이 위치한 'yolo' 폴더 기준
# z_fruit_quality_project/fruits/yolo/
NOTEBOOK_DIR = Path.cwd() 

# /z_fruit_quality_project/fruits/
FRUITS_DIR = NOTEBOOK_DIR.parent 

# /z_fruit_quality_project/
ROOT_PROJECT_DIR = FRUITS_DIR.parent 

# --- 2. 원본 데이터 경로 ---
# /z_fruit_quality_project/fruits/datasets/
DATASET_DIR = FRUITS_DIR / "datasets"
JSON_DIR = DATASET_DIR / "json_labels"
IMAGE_DIR = DATASET_DIR / "images"

# --- 3. YOLOv5 레포지토리 경로 ---
# /z_fruit_quality_project/yolov5/
YOLOV5_DIR = ROOT_PROJECT_DIR / "yolov5"

# --- 4. YOLO 학습용 데이터 생성 경로 ---
# /z_fruit_quality_project/fruits/datasets/yolo_data/
# 여기에 images/train, images/val, labels/train, labels/val 등이 생성됩니다.
YOLO_DATA_DIR = DATASET_DIR / "yolo_data"

# --- 5. 클래스 및 경로 생성 ---
# JSON의 'cate3' 값과 실제 등급명을 매핑합니다.
GRADE_MAP = {
    "특": "특상",
    "상": "상",
    "중": "중"
}

# 프로젝트의 최종 9개 클래스를 정의합니다. (순서가 중요합니다!)
CLASSES = [
    f"{fruit}-{grade}"
    for fruit in ["사과", "배", "감"]
    for grade in ["특상", "상", "중"]
]
# {'사과-특상': 0, '사과-상': 1, ... '감-중': 8} 형태의 딕셔너리 생성
CLASS_TO_ID = {name: i for i, name in enumerate(CLASSES)}

print(f"✅ 기본 경로 설정 완료. YOLOv5 경_로: {YOLOV5_DIR}")
print(f"✅ 총 {len(CLASSES)}개 클래스 정의 완료:")
print(CLASS_TO_ID)

# 필요한 디렉토리 생성
os.makedirs(YOLO_DATA_DIR / "images" / "train", exist_ok=True)
os.makedirs(YOLO_DATA_DIR / "images" / "val", exist_ok=True)
os.makedirs(YOLO_DATA_DIR / "labels" / "train", exist_ok=True)
os.makedirs(YOLO_DATA_DIR / "labels" / "val", exist_ok=True)

✅ 기본 경로 설정 완료. YOLOv5 경_로: /Users/handaeseong/dev/data-engineer/workspaces/z_fruit_quality_project/yolov5
✅ 총 9개 클래스 정의 완료:
{'사과-특상': 0, '사과-상': 1, '사과-중': 2, '배-특상': 3, '배-상': 4, '배-중': 5, '감-특상': 6, '감-상': 7, '감-중': 8}


In [2]:
def convert_to_yolo_format(json_path, class_map, grade_map):
    """
    AI Hub JSON 파일을 읽어 YOLO 포맷의 문자열 리스트를 반환합니다.
    (하나의 JSON에 여러 객체가 있을 경우를 대비해 리스트로 반환)
    """
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error reading {json_path}: {e}")
        return []

    yolo_labels = []

    # 이미지 크기
    img_width = float(data.get("img_width", 1000)) # 기본값 1000
    img_height = float(data.get("img_height", 1000)) # 기본값 1000

    # 클래스 ID 가져오기
    cate1 = data.get("cate1") # 예: "사과"
    cate3 = data.get("cate3") # 예: "특"
    
    if not cate1 or not cate3:
        return []

    # 등급 매핑 (예: "특" -> "특상")
    grade = grade_map.get(cate3)
    if not grade:
        print(f"Warning: Unknown grade '{cate3}' in {json_path}")
        return []

    # 최종 클래스 이름 (예: "사과-특상")
    class_name = f"{cate1}-{grade}"
    
    # 클래스 ID 조회
    class_id = class_map.get(class_name)
    if class_id is None:
        print(f"Warning: Unknown class name '{class_name}' in {json_path}")
        return []

    # 바운딩 박스 좌표 (Absolute pixels)
    # AI Hub 데이터는 종종 이미지 전체가 BBox일 수 있습니다.
    bbox = data.get("bndbox")
    if not bbox:
        return []

    xmin = float(bbox["xmin"])
    ymin = float(bbox["ymin"])
    xmax = float(bbox["xmax"])
    ymax = float(bbox["ymax"])

    # YOLO 포맷으로 변환 (Normalized center x, center y, width, height)
    x_center = ((xmin + xmax) / 2) / img_width
    y_center = ((ymin + ymax) / 2) / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height

    # 0~1 사이 값으로 클리핑 (좌표가 이미지 밖으로 나가는 경우 방지)
    x_center = max(0.0, min(1.0, x_center))
    y_center = max(0.0, min(1.0, y_center))
    width = max(0.0, min(1.0, width))
    height = max(0.0, min(1.0, height))

    # YOLO .txt 파일 형식: "class_id x_center y_center width height"
    yolo_labels.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return yolo_labels

print("✅ YOLO 변환 함수 정의 완료.")

✅ YOLO 변환 함수 정의 완료.


In [3]:
print("🔄 데이터 변환 및 분할 시작...")

# 원본 이미지/JSON 파일 목록 가져오기
json_files = sorted(glob(str(JSON_DIR / "*.json")))
image_files = sorted(glob(str(IMAGE_DIR / "*.png"))) # .jpg 등 다른 형식이면 수정

# 파일명 기준으로 매칭하기 위해 파일명(확장자 제외)만 추출
json_basenames = {Path(f).stem: f for f in json_files}
image_basenames = {Path(f).stem: f for f in image_files}

# 두 목록에 모두 존재하는 파일명(basename)만 선별
matched_basenames = sorted(list(json_basenames.keys() & image_basenames.keys()))

print(f"총 {len(matched_basenames)}개의 이미지/JSON 쌍을 찾았습니다.")

# 데이터 랜덤 셔플
random.seed(42) # 재현성을 위해 시드 고정
random.shuffle(matched_basenames)

# Train/Val 분할 (예: 80% train, 20% val)
split_index = int(len(matched_basenames) * 0.8)
train_basenames = matched_basenames[:split_index]
val_basenames = matched_basenames[split_index:]

print(f"  - Train 세트: {len(train_basenames)}개")
print(f"  - Val 세트: {len(val_basenames)}개")

# 파일 복사 및 라벨 생성 함수
def process_and_copy_files(basenames, split_name): # split_name: "train" or "val"
    count = 0
    for basename in basenames:
        json_path = json_basenames[basename]
        image_path = image_basenames[basename]
        
        # 1. YOLO 라벨 변환
        yolo_labels = convert_to_yolo_format(json_path, CLASS_TO_ID, GRADE_MAP)
        
        if not yolo_labels:
            print(f"Skipping {basename}: No valid labels found.")
            continue
            
        # 2. 라벨 파일(.txt) 저장
        label_filename = YOLO_DATA_DIR / "labels" / split_name / f"{basename}.txt"
        with open(label_filename, 'w', encoding='utf-8') as f:
            f.write("\n".join(yolo_labels))
            
        # 3. 이미지 파일 복사
        image_filename = YOLO_DATA_DIR / "images" / split_name / Path(image_path).name
        shutil.copy(image_path, image_filename)
        count += 1
    
    print(f"✅ {split_name} 세트 처리 완료: {count}개 파일")

# 실행
process_and_copy_files(train_basenames, "train")
process_and_copy_files(val_basenames, "val")

print("\n🎉 데이터 전처리 및 분할 완료!")

🔄 데이터 변환 및 분할 시작...
총 70개의 이미지/JSON 쌍을 찾았습니다.
  - Train 세트: 56개
  - Val 세트: 14개
Skipping persimmon_booyu_S_76-31_1TOP: No valid labels found.
Skipping apple_fuji_S_26-56: No valid labels found.
Skipping pear_chuhwang_S_25-90: No valid labels found.
Skipping apple_fuji_S_26-54: No valid labels found.
Skipping persimmon_booyu_S_76-27_2FR45: No valid labels found.
Skipping apple_fuji_S_26-59: No valid labels found.
Skipping apple_fuji_S_26-63: No valid labels found.
Skipping pear_chuhwang_S_25-84: No valid labels found.
Skipping apple_fuji_S_26-57: No valid labels found.
Skipping persimmon_booyu_S_76-26_1TOP: No valid labels found.
Skipping persimmon_booyu_S_76-33_3FR90: No valid labels found.
Skipping pear_chuhwang_S_25-87: No valid labels found.
Skipping apple_fuji_S_26-58: No valid labels found.
Skipping pear_chuhwang_S_25-81: No valid labels found.
Skipping apple_fuji_S_26-55: No valid labels found.
Skipping persimmon_booyu_S_76-29_4DI45: No valid labels found.
Skipping persimmon_booy

In [4]:
# .yaml 파일에 쓸 내용 정의
# 경로는 yolov5/ 디렉토리 기준의 상대 경로여야 합니다.
# 우리 데이터는 yolov5/../fruits/datasets/yolo_data/ 에 있습니다.
# 따라서 ../fruits/datasets/yolo_data 로 설정합니다.

yaml_content = f"""
# Train/Val 데이터 경로
path: {YOLO_DATA_DIR.resolve()}  # 데이터셋 루트 디렉토리 (절대 경로)
train: images/train  # train images (path 기준 상대 경로)
val: images/val    # val images (path 기준 상대 경로)

# 클래스 정보
nc: {len(CLASSES)}  # 클래스 개수
names: {CLASSES} # 클래스 이름 목록
"""

# .yaml 파일을 yolov5/data/ 폴더에 저장
# YOLOv5는 기본적으로 yolov5/data/ 경로에서 .yaml 파일을 찾습니다.
YAML_FILE_PATH = YOLOV5_DIR / "data" / "fruit_quality.yaml"
os.makedirs(YOLOV5_DIR / "data", exist_ok=True)

with open(YAML_FILE_PATH, 'w', encoding='utf-8') as f:
    f.write(yaml_content)

print(f"✅ 데이터셋 YAML 파일 생성 완료: {YAML_FILE_PATH}")
print("\n--- YAML 내용 미리보기 ---")
print(yaml_content)

✅ 데이터셋 YAML 파일 생성 완료: /Users/handaeseong/dev/data-engineer/workspaces/z_fruit_quality_project/yolov5/data/fruit_quality.yaml

--- YAML 내용 미리보기 ---

# Train/Val 데이터 경로
path: /Users/handaeseong/dev/data-engineer/workspaces/z_fruit_quality_project/fruits/datasets/yolo_data  # 데이터셋 루트 디렉토리 (절대 경로)
train: images/train  # train images (path 기준 상대 경로)
val: images/val    # val images (path 기준 상대 경로)

# 클래스 정보
nc: 9  # 클래스 개수
names: ['사과-특상', '사과-상', '사과-중', '배-특상', '배-상', '배-중', '감-특상', '감-상', '감-중'] # 클래스 이름 목록



In [5]:
# %cd: 주피터 노트북의 현재 작업 디렉토리를 변경합니다.
%cd {YOLOV5_DIR}

print(f"현재 디렉토리: {Path.cwd()}")
print("YOLOv5m 모델 학습을 시작합니다...")

# ! (느낌표) : 셸 명령어 실행
# --data 경로는 yolov5/data/fruit_quality.yaml을 의미합니다.
!python train.py --img 640 --batch 16 --epochs 100 --data fruit_quality.yaml --weights yolov5m.pt --name fruit_quality_yolov5m

# 학습 완료 후 원래 노트북 디렉토리로 복귀 (선택 사항)
%cd {NOTEBOOK_DIR}
print(f"학습 완료. 현재 디렉토리: {Path.cwd()}")

/Users/handaeseong/dev/data-engineer/workspaces/z_fruit_quality_project/yolov5
현재 디렉토리: /Users/handaeseong/dev/data-engineer/workspaces/z_fruit_quality_project/yolov5
YOLOv5m 모델 학습을 시작합니다...
train: weights=yolov5m.pt, cfg=, data=fruit_quality.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=fruit_quality_yolov5m, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
requirements: Ultralytics requirement ['th